In [1]:
# Create sub holding key.
"""
------------------------
Creating an API Factory:
------------------------

The snippet below loads LUSID and other packages needed for the snippet examples.

We also create an API factory for interacting with the LUSID's API methods.

"""

# Set up LUSID
import os
import pandas as pd
import json
import uuid
import pytz
from IPython.core.display import HTML
from datetime import datetime, timedelta, timezone
import pprint as pp

import logging
logging.basicConfig(level=logging.INFO)

# Import LUSID SDK
import lusid
import lusid.models as models

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from lusidtools.lpt.lpt import to_date

# Set pandas display options
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.2f}".format

# Authenticate to SDK
# Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Run the Notebook locally using a secrets file (see https://support.lusid.com/knowledgebase/article/KA-01663)
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="VSCode"
)

print('LUSID Environment Initialised')
print('API Version: ', api_factory.build(
    lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

# Define a portfolio and parameters.
portfolio_code = "JC_UK"
portfolio_name = "Joe Cockings Demo Portfolio"
scope = "Demo2"

domain = "Portfolio"
sub_holding_key_code = "strategy"

portfolio_sub_holding_keys = [f"Transaction/{scope}/{sub_holding_key_code}"]

# Create a new sub-holding key via API.
property_definition_api = api_factory.build(lusid.api.PropertyDefinitionsApi)

# Attempt to create a SHK definition.
try:
    create_shk = property_definition_api.create_property_definition(
        create_property_definition_request = models.CreatePropertyDefinitionRequest(
            domain = "Transaction",
            scope = scope,
            code = "strategy",
            display_name = "Strategy",
            data_type_id = models.ResourceId(
                scope = "system",
                code = "string",
            ),
            life_time = "Perpetual",
            property_description = "Strategy of the trade",
        )
    )
    
    print(create_shk)

except lusid.ApiException as e:
    print(e)
    print(json.loads(e.body)["title"])


# Define instrument property for moodys rating.
inst_prop_rating_code = 'MoodyRating'

try:
    create_inst_prop = property_definition_api.create_property_definition(
        create_property_definition_request = models.CreatePropertyDefinitionRequest(
            domain = "Instrument",
            scope = scope,
            code = inst_prop_rating_code,
            display_name = "Moody's Rating",
            data_type_id = models.ResourceId(
                scope = "system",
                code = "number",
            ),
            life_time = "TimeVariant",
            property_description = "Moody's rating of the instrument",
        )
    )
    
    print(create_inst_prop)

except lusid.ApiException as e:
    print(e)
    print(json.loads(e.body)["title"])

LUSID Environment Initialised
API Version:  0.6.13127.0
{'code': 'strategy',
 'collection_type': None,
 'constraint_style': 'Property',
 'data_type_id': {'code': 'string', 'scope': 'system'},
 'derivation_formula': None,
 'display_name': 'Strategy',
 'domain': 'Transaction',
 'href': None,
 'key': 'Transaction/Demo2/strategy',
 'life_time': 'Perpetual',
 'links': [{'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/schemas/entities/PropertyDefinition',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-joe.lusid.com/app/insights/logs/0HN49EQTFEAMS:0000003B',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'properties': {},
 'property_definition_type': 'ValueProperty',
 'property_description': 'Strategy of the trade',
 'scope': 'Demo2',
 'type': 'Label'

In [2]:
# Create portfolio.

# Create a Transactions Portfolios API
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

# Attempt to create a Transaction Portfolio
try:
    create_portfolio_response = transaction_portfolio_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name=portfolio_name,
            code=portfolio_code,
            description="Joe's demo portfolio.",
            base_currency="GBP",
            created="2020-01-01",
            sub_holding_keys=portfolio_sub_holding_keys,
            #properties=transaction_portfolio_properties,
            instrument_scopes = [
                scope
            ],
        ),
    )

    print(create_portfolio_response)

except lusid.ApiException as e:
    print(json.loads(e.body)["title"])

{'accounting_method': 'Default',
 'amortisation_method': 'NoAmortisation',
 'amortisation_rule_set_id': None,
 'base_currency': 'GBP',
 'cash_gain_loss_calculation_date': 'Default',
 'created': datetime.datetime(2020, 1, 1, 0, 0, tzinfo=tzutc()),
 'description': "Joe's demo portfolio.",
 'display_name': 'Joe Cockings Demo Portfolio',
 'href': 'https://fbn-joe.lusid.com/api/api/portfolios/Demo2/JC_UK?asAt=2024-06-11T13%3A09%3A52.7113100%2B00%3A00',
 'id': {'code': 'JC_UK', 'scope': 'Demo2'},
 'instrument_event_configuration': {'recipe_id': None,
                                    'transaction_template_scopes': ['default']},
 'instrument_scopes': ['Demo2'],
 'is_derived': False,
 'links': [{'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/portfolios/Demo2/JC_UK/properties?effectiveAt=2020-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2024-06-11T13%3A09%3A52.7113100%2B00%3A00',
            'method': 'GET',
            'relation': 'Properties'},
           {'descrip

In [3]:
# Add instruments to the domain, in the demo scope.

# First generate distinct instruments from the transactions file.

# Create instruments API
instrument_api = api_factory.build(lusid.api.InstrumentsApi)

instrument_keys = [
    "ticker",
    "sedol",
    "instrument_type",
    "instrument_id",
    "name",
    "currency",
]

instrument_identifiers = {
    "ticker": "Isin",
    "sedol": "Sedol",
    "instrument_id": "ClientInternal",
}

instrument_type_identifiers = {
    "equity": "Equity",
    "bond": "Bond"
}

# function to return formatted datetime
def formatted_datetime(date_string):
    day, month, year = date_string.split("/")
    formatted_date = datetime(int(year), int(month), int(day), tzinfo=timezone.utc)
    return formatted_date.isoformat()

def resolve_lusid_instrument_definition(instrument):
    match instrument["instrument_type"]:
        case "equity":
            return models.Equity(
                dom_ccy=instrument["currency"],
                instrument_type=instrument_type_identifiers[instrument["instrument_type"]],
            )
        case "bond":
            return models.Bond(
                dom_ccy = instrument["currency"],
                instrument_type = instrument_type_identifiers[instrument["instrument_type"]],
                start_date = formatted_datetime(instrument["start_date"]),
                maturity_date = formatted_datetime(instrument["maturity_date"]),
                principal = instrument["principal"],
                coupon_rate = float(instrument["coupon"]),
                
                # Default flow conventions
                flow_conventions = models.FlowConventions(
                    currency = instrument["currency"],
                    payment_frequency = "6M",
                    roll_convention = "F",
                    day_count_convention="Actual360",
                    payment_calendars=[],
                    reset_calendars=[]
                )
            )

# define request body dictionary object.
instrument_definitions_request_body = {}

# Read instruments form file into df.
instruments = pd.read_csv('instruments.csv')

#Generate request body.
for enum, instrument in instruments.iterrows():
    
    identifiers = {}

    #Iterate over each instrument key
    for client_ref, lusid_ref in instrument_identifiers.items():
        identifiers[lusid_ref] = models.InstrumentIdValue(
            value=instrument[client_ref],
        )

    instrumentName = instrument["name"]
    instrument_definitions_request_body[f"INS_REQ_{enum}"] = models.InstrumentDefinition(
        name=instrument["name"],
        identifiers=identifiers,
        properties=[
            models.ModelProperty(
                key = f"Instrument/default/Isin",
                value = models.PropertyValue(
                    label_value = instrument["ticker"]
                ),
            ),
            models.ModelProperty(
                key = f"Instrument/default/Sedol",
                value = models.PropertyValue(
                    label_value = instrument["sedol"]
                ),
            ),
            models.ModelProperty(
                key = f"Instrument/{scope}/{inst_prop_rating_code}",
                value = models.PropertyValue(
                    metric_value = models.MetricValue(
                        value = float(instrument["moody"])
                        )
                ),
            )
        ],
        definition=resolve_lusid_instrument_definition(instrument),
    )

print("Now attempting to upsert to LUSID")

# Attempt to upsert the instrument definitions.
try:
    upsert_instruments_response = instrument_api.upsert_instruments(
        scope=scope,
        request_body=instrument_definitions_request_body,
    )

    print(upsert_instruments_response)

except lusid.ApiException as e:
    pp.pprint(f'Exception thrown: {e}')
    pp.pprint(json.loads(e.body)["title"])

Now attempting to upsert to LUSID
{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/schemas/entities/UpsertInstrumentsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-joe.lusid.com/app/insights/logs/0HN4A2M0N8RG9:00000049',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'metadata': {'actions': [{'description': 'The request identifiers of Created Instruments',
 'identifier_type': 'RequestId',
 'identifiers': ['INS_REQ_3',
                 'INS_REQ_2',
                 'INS_REQ_0',
                 'INS_REQ_7',
                 'INS_REQ_6',
                 'INS_REQ_1',
                 'INS_REQ_4',
                 'INS_REQ_8',
                 'INS_REQ_10',
                 'INS_REQ_9',
         

In [4]:
# Add transactions.

portfolio_sub_holding_key = portfolio_sub_holding_keys[0]

transactions = pd.read_csv('transactions.csv')

# Generate list of transaction requests.
security_transaction_types = [
    "Buy",
    "Sell",
    "SellShort",
    "Cover"
]

cashflow_transaction_types = [
    "FundsIn",
    "FundsOut"
]

# function to return formatted datetime
def formatted_datetime(date_string):
    day, month, year = date_string.split("/")
    formatted_date = datetime(int(year), int(month), int(day), tzinfo=timezone.utc)
    return formatted_date.isoformat()

# function to resolve correct identifier
def resolve_instrument_identifier(transaction):
    transaction_type = transaction["txn_type"]
    if transaction_type in security_transaction_types: return "Isin"
    elif transaction_type in cashflow_transaction_types: return "Currency"
    # Should crash otherwise.

# function to resolve correct instrument scope
def resolve_instrument_scope(transaction):
    transaction_type = transaction["txn_type"]
    if transaction_type in security_transaction_types: return scope
    elif transaction_type in cashflow_transaction_types: return "default"
    # Should crash otherwise.

def resolve_transaction_price_type(transaction):
    transaction_type = transaction["txn_type"]
    if transaction_type in security_transaction_types:
        if transaction["instrument_type"] == "equity": return "Price"
        if transaction["instrument_type"] == "bond": return "Price"    
    elif transaction_type in cashflow_transaction_types: return "CashFlowPerUnit"
    # Should crash otherwise.

transaction_requests = []

for transaction_number, transaction in transactions.iterrows():
    transactionID = transaction["txn_id"]
    print(f"Creating TransactionRequest object for Transaction ID {transactionID}")
    transaction_requests.append(
        models.TransactionRequest(
            transaction_id = transaction["txn_id"],
            type = transaction["txn_type"],
            instrument_identifiers = {
                f"Instrument/default/{resolve_instrument_identifier(transaction)}": transaction["ticker"],
            },
            transaction_date = formatted_datetime(transaction["txn_trade_date"]),
            settlement_date = formatted_datetime(transaction["txn_settle_date"]),
            units = float(transaction["txn_units"]),
            transaction_price = models.TransactionPrice(
                price = float(transaction["txn_price"]),
                type = resolve_transaction_price_type(transaction)
            ),
            total_consideration = models.CurrencyAndAmount(
                amount = float(transaction["txn_consideration"]),
                currency = transaction["currency"],
            ),
            transaction_currency = transaction["currency"],
            properties = {
                portfolio_sub_holding_key: models.PerpetualProperty(
                    key = portfolio_sub_holding_key,
                    value = models.PropertyValue(
                        label_value = transaction[sub_holding_key_code]
                    ),
                ),
            }
        )
    )

print(transaction_requests)

# Create a Transactions Portfolios API
transaction_portfolio_api = api_factory.build(lusid.api.TransactionPortfoliosApi)

try:
    upsert_transaction = transaction_portfolio_api.upsert_transactions(
        scope=scope, # Scope of the portfolio.
        code=portfolio_code, # Code of the portfolio.
        transaction_request=transaction_requests,
    )

    print(upsert_transaction)

except lusid.ApiException as e:
    pp.pprint(e)
    pp.pprint(json.loads(e.body)["title"])


Creating TransactionRequest object for Transaction ID trd_0001
Creating TransactionRequest object for Transaction ID trd_0002
Creating TransactionRequest object for Transaction ID trd_0003
Creating TransactionRequest object for Transaction ID trd_0004
Creating TransactionRequest object for Transaction ID trd_0005
Creating TransactionRequest object for Transaction ID trd_0006
Creating TransactionRequest object for Transaction ID trd_0007
Creating TransactionRequest object for Transaction ID trd_0008
Creating TransactionRequest object for Transaction ID trd_0009
Creating TransactionRequest object for Transaction ID trd_0010
Creating TransactionRequest object for Transaction ID trd_0011
Creating TransactionRequest object for Transaction ID trd_0012
Creating TransactionRequest object for Transaction ID trd_0022
Creating TransactionRequest object for Transaction ID trd_0014
Creating TransactionRequest object for Transaction ID trd_0015
Creating TransactionRequest object for Transaction ID t

In [5]:
# Upsert quotes.

quote_provider = "Client"
quote_source = quote_provider

symbology = "Isin"

# function to return formatted datetime
def formatted_datetime(date_string):
    formatted_date = datetime.strptime(date_string, '%d-%b-%y')
    formatted_date = formatted_date.replace(tzinfo=timezone.utc)

    return formatted_date.isoformat()

# Create an API factory for interacting with the LUSID APIs
quotes_api = api_factory.build(lusid.api.QuotesApi)

# Read quotes from csv.
quotes = pd.read_csv('quotes.csv')

quote_fields = [
    "bid",
    "mid",
    "ask",
]

quote_upsert = {}

for row_index, quote in quotes.iterrows():
    for quote_field in quote_fields:
        quote_upsert["QUOTE_" + quote_field + "_" + str(row_index)] = models.UpsertQuoteRequest(
            quote_id = models.QuoteId(
                quote_series_id = models.QuoteSeriesId(
                    provider = quote_provider,
                    price_source = quote_source,
                    instrument_id = quote["ISIN"],
                    instrument_id_type = symbology,
                    quote_type = "Price",
                    field = quote_field,
                ),
                effective_at = formatted_datetime(quote["quote_date"])
            ),
            metric_value = models.MetricValue(
                value = float(quote[quote_field]),
                unit = "GBP",
            )
        )

print(f'Quote upsert is :')
for value in quote_upsert.values():
    print(value.to_str())
    
print(quote_upsert)

try:
    quote_upsert_response = quotes_api.upsert_quotes(
        scope = scope,
        request_body = quote_upsert,
    )

    print(quote_upsert_response)

except Exception as e:
    print(f'Thrown exception: {e}')

Quote upsert is :
{'lineage': None,
 'metric_value': {'unit': 'GBP', 'value': 2.29},
 'quote_id': {'effective_at': '2024-06-09T00:00:00+00:00',
              'quote_series_id': {'field': 'bid',
                                  'instrument_id': 'GB0002162385',
                                  'instrument_id_type': 'Isin',
                                  'price_source': 'Client',
                                  'provider': 'Client',
                                  'quote_type': 'Price'}},
 'scale_factor': None}
{'lineage': None,
 'metric_value': {'unit': 'GBP', 'value': 2.3},
 'quote_id': {'effective_at': '2024-06-09T00:00:00+00:00',
              'quote_series_id': {'field': 'mid',
                                  'instrument_id': 'GB0002162385',
                                  'instrument_id_type': 'Isin',
                                  'price_source': 'Client',
                                  'provider': 'Client',
                                  'quote_type': 'Price'

In [6]:
# Upload pricing curve

def formatted_datetime(date_string):
    day, month, year = date_string.split("/")
    formatted_date = datetime(int(year), int(month), int(day), tzinfo=timezone.utc)
    return formatted_date.isoformat()

complex_md_api = api_factory.build(lusid.ComplexMarketDataApi)


with open('curve.json') as curve_data_file:
    curve_data = json.load(curve_data_file)

pp.pprint(curve_data)

discount_rates = pd.read_csv('discount_rate_curve.csv')


for discount_rate_index, discount_rate in discount_rates.iterrows():
    curve_data["GBP/GBPOIS"]["marketData"]["dates"].append(formatted_datetime(discount_rate["date"]))
    curve_data["GBP/GBPOIS"]["marketData"]["discountFactors"].append(float(discount_rate["factor"]))

pp.pprint(curve_data)

request_body = curve_data

try:
    upload_curve = complex_md_api.upsert_complex_market_data(scope, request_body)

    print(upload_curve)

except Exception as e:
    print(f'Thrown exception: {e}')


{'GBP/GBPOIS': {'marketData': {'baseDate': '2024-06-10T00:00:00.0000000+00:00',
                               'dates': [],
                               'discountFactors': [],
                               'lineage': 'SomeLineage',
                               'marketDataOptions': {'backExtrapolationType': 'None',
                                                     'dayCountConvention': 'Actual365',
                                                     'frontExtrapolationType': 'Linear',
                                                     'marketDataOptionsType': 'CurveOptions'},
                               'marketDataType': 'DiscountFactorCurveData'},
                'marketDataId': {'effectiveAt': '2024-06-10T00:00:00.0000000+00:00',
                                 'marketAsset': 'GBP/GBPOIS',
                                 'priceSource': 'Client',
                                 'provider': 'Client'}}}
{'GBP/GBPOIS': {'marketData': {'baseDate': '2024-06-10T00:00:00.0000

In [8]:
#upsert recipe using market pricings

recipe_code = "standard-recipe-marketprice"

# Create an API factory for interacting with the LUSID APIs
config_recipe_api = api_factory.build(lusid.ConfigurationRecipeApi)

with open('market_recipe.json') as discount_recipe_file:
    recipe_request = json.load(discount_recipe_file)
pp.pprint(recipe_request)

upsert_recipe_request = recipe_request

try:
    recipe_upsert_response = config_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request,
    )

    print(recipe_upsert_response)

except Exception as e:
    print(f'Thrown exception: {e}')



    #metadata mismatch Dim:0,Units:(LUID_00003D8X) != Dim:0,Units:(CCY_GBP).
#metadata mismatch Dim:0,Units:(LUID_00003D8F) != Dim:0,Units:(LUID_00003D8D).

{'configurationRecipe': {'aggregation': {'options': {'allowPartialEntitlementSuccess': False,
                                                     'applyIso4217Rounding': False,
                                                     'useAnsiLikeSyntax': False}},
                         'code': 'standard-recipe',
                         'description': 'Standard recipe configuration',
                         'holding': {'taxLotLevelHoldings': True},
                         'market': {'groupedMarketRules': [],
                                    'marketRules': [{'dataScope': 'Demo2',
                                                     'field': 'mid',
                                                     'key': 'Quote.Isin.*',
                                                     'priceSource': 'Client',
                                                     'quoteType': 'Price',
                                                     'sourceSystem': 'Lusid',
                                  

In [37]:
#upsert recipe using market pricing and CTVOM discounting.

# Create an API factory for interacting with the LUSID APIs
config_recipe_api = api_factory.build(lusid.ConfigurationRecipeApi)

with open('discount_recipe.json') as discount_recipe_file:
    recipe_request = json.load(discount_recipe_file)
pp.pprint(recipe_request)

upsert_recipe_request = recipe_request

try:
    recipe_upsert_response = config_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request,
    )

    print(recipe_upsert_response)

except Exception as e:
    print(f'Thrown exception: {e}')


{'configurationRecipe': {'aggregation': {'options': {'allowPartialEntitlementSuccess': False,
                                                     'applyIso4217Rounding': False,
                                                     'useAnsiLikeSyntax': False}},
                         'code': 'standard-recipe-discounting',
                         'description': 'Standard recipe configuration',
                         'holding': {'taxLotLevelHoldings': True},
                         'market': {'groupedMarketRules': [],
                                    'marketRules': [{'dataScope': 'Demo',
                                                     'field': 'mid',
                                                     'key': 'Quote.Isin.*',
                                                     'priceSource': 'Client',
                                                     'quoteType': 'Price',
                                                     'sourceSystem': 'Lusid',
                       

In [9]:
# Define a reference portfolio.

def formatted_datetime(date_string):
    formatted_date = datetime.strptime(date_string, '%d-%b-%y')
    formatted_date = formatted_date.replace(tzinfo=timezone.utc)

    return formatted_date.isoformat()

reference_portfolio_code = 'SP500'
reference_portfolio_description =  "S&P 500 Index"
reference_portfolio_display_name = "S&P 500 Tracker"

# Create an API for interacting with the LUSID APIs
reference_portfolio_api = api_factory.build(lusid.ReferencePortfolioApi)

try:
    reference_portfolio_creation_response = reference_portfolio_api.create_reference_portfolio(
        scope = scope,
        create_reference_portfolio_request = models.CreateReferencePortfolioRequest(
            display_name = reference_portfolio_display_name,
            description = reference_portfolio_description,
            code=reference_portfolio_code,
            created=formatted_datetime('05-Jun-10'),
        )
    )

    print(reference_portfolio_creation_response)

except Exception as e:
    pp.pprint(f'Thrown exception {e}')





{'accounting_method': 'Default',
 'amortisation_method': 'NoAmortisation',
 'amortisation_rule_set_id': None,
 'base_currency': 'ZZZ',
 'cash_gain_loss_calculation_date': 'Default',
 'created': datetime.datetime(2010, 6, 5, 0, 0, tzinfo=tzutc()),
 'description': 'S&P 500 Index',
 'display_name': 'S&P 500 Tracker',
 'href': 'https://fbn-joe.lusid.com/api/api/portfolios/Demo2/SP500?effectiveAt=2010-06-05T00%3A00%3A00.0000000%2B00%3A00&asAt=2024-06-11T13%3A46%3A05.1802320%2B00%3A00',
 'id': {'code': 'SP500', 'scope': 'Demo2'},
 'instrument_event_configuration': {'recipe_id': None,
                                    'transaction_template_scopes': ['default']},
 'instrument_scopes': [],
 'is_derived': False,
 'links': [{'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/referenceportfolios/Demo2/SP500/constituents?asAt=2024-06-11T13%3A46%3A05.1802320%2B00%3A00',
            'method': 'GET',
            'relation': 'Constituents'},
           {'description': None,
 

In [10]:
# Add instrument defs of constituent portfolio to lusid.

instrument_keys = [
    "ticker",
    "sedol",
    "instrument_type",
    "instrument_id",
    "name",
    "currency",
]

instrument_identifiers = {
    "ticker": "Isin",
    "instrument_id": "ClientInternal",
}

instrument_type_identifiers = {
    "equity": "Equity",
    "bond": "Bond"
}

# function to return formatted datetime
def formatted_datetime(date_string):
    day, month, year = date_string.split("/")
    formatted_date = datetime(int(year), int(month), int(day), tzinfo=timezone.utc)
    return formatted_date.isoformat()

def resolve_lusid_instrument_definition(instrument):
    match instrument["instrument_type"]:
        case "equity":
            return models.Equity(
                dom_ccy=instrument["currency"],
                instrument_type=instrument_type_identifiers[instrument["instrument_type"]],
            )
        case "bond":
            return models.Bond(
                dom_ccy = instrument["currency"],
                instrument_type = instrument_type_identifiers[instrument["instrument_type"]],
                start_date = formatted_datetime(instrument["start_date"]),
                maturity_date = formatted_datetime(instrument["maturity_date"]),
                principal = instrument["principal"],
                coupon_rate = float(instrument["coupon"]),
                
                # Default flow conventions
                flow_conventions = models.FlowConventions(
                    currency = instrument["currency"],
                    payment_frequency = "6M",
                    roll_convention = "F",
                    day_count_convention="Actual360",
                    payment_calendars=[],
                    reset_calendars=[]
                )
            )

# define request body dictionary object.
instrument_definitions_request_body = {}

instruments = pd.read_csv('sp500constituents.csv')

#Generate request body.
for enum, instrument in instruments.iterrows():
    
    identifiers = {}

    #Iterate over each instrument key
    for client_ref, lusid_ref in instrument_identifiers.items():
        identifiers[lusid_ref] = models.InstrumentIdValue(
            value=instrument[client_ref],
        )
        #print(f"instrument_identifier {instrument_identifier} set as.")
        #print(instrument[instrument_identifier])
        #print("with type")
        #print(type(instrument[instrument_identifier]))


    instrumentName = instrument["name"]
    #print(f"name of instrument in loop is {instrumentName} with type {type(instrumentName)})")
    #print("Instrument name is")
    #print(instrument["name"])
    instrument_definitions_request_body[f"INS_REQ_{enum}"] = models.InstrumentDefinition(
        name=instrument["name"],
        identifiers=identifiers,
        properties=[
            models.ModelProperty(
                key = f"Instrument/default/Isin",
                value = models.PropertyValue(
                    label_value = instrument["ticker"]
                ),
            )
        ],
        definition=resolve_lusid_instrument_definition(instrument),
    )


#print(instrument_definitions_request_body)

print("Now attempting to upsert to LUSID")

# Attempt to upsert the instrument definitions.
try:
    upsert_instruments_response = instrument_api.upsert_instruments(
        scope=scope,
        request_body=instrument_definitions_request_body,
    )

    print(upsert_instruments_response)

except lusid.ApiException as e:
    pp.pprint(f'Exception thrown: {e}')
    pp.pprint(json.loads(e.body)["title"])

Now attempting to upsert to LUSID
{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/schemas/entities/UpsertInstrumentsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-joe.lusid.com/app/insights/logs/0HN49TF02OGJM:0000002C',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'metadata': {'actions': [{'description': 'The request identifiers of Created Instruments',
 'identifier_type': 'RequestId',
 'identifiers': ['INS_REQ_0', 'INS_REQ_1', 'INS_REQ_2'],
 'type': 'CreatedInstruments'}]},
 'values': {'INS_REQ_0': {'asset_class': 'Equities',
                          'dom_ccy': 'GBP',
                          'href': 'https://fbn-joe.lusid.com/api/api/instruments/LusidInstrumentId/LUID_00003D9K?scope=Dem

In [15]:
# Set up LUSID
import os
import pandas as pd
import json
import uuid
import pytz
from IPython.core.display import HTML
from datetime import datetime, timedelta, timezone
import pprint as pp

import logging
logging.basicConfig(level=logging.INFO)

# Import LUSID SDK
import lusid
import lusid.models as models

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from lusidtools.lpt.lpt import to_date
# Add constituents to ref portfolio.
def formatted_datetime(date_string):
    formatted_date = datetime.strptime(date_string, '%d-%b-%y')
    formatted_date = formatted_date.replace(tzinfo=timezone.utc)

    return formatted_date.isoformat()


ref_portfolio_constituents = pd.read_csv('sp500constituents.csv')

reference_portfolio_constituents = []

for index, constituent in ref_portfolio_constituents.iterrows():
    reference_portfolio_constituents.append(
        models.ReferencePortfolioConstituentRequest(
            instrument_identifiers = {
                'Instrument/default/Isin': constituent["ticker"]
            },
            weight = constituent["Weighting"],
            currency=constituent["currency"]
        )
    )

upsert_reference_portfolio_constituents_response = reference_portfolio_api.upsert_reference_portfolio_constituents(
    scope = scope,
    code = reference_portfolio_code,
    upsert_reference_portfolio_constituents_request = models.UpsertReferencePortfolioConstituentsRequest(
        effective_from = formatted_datetime('05-Jun-24'),
        weight_type='Static',
        constituents=reference_portfolio_constituents
    )
)

print(upsert_reference_portfolio_constituents_response)





{'href': None,
 'links': [{'description': None,
            'href': 'https://fbn-joe.lusid.com/api/api/schemas/entities/ReferencePortfolioConstituentRequest',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://fbn-joe.lusid.com/app/insights/logs/0HN49EJ19VIIM:000001B7',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'version': None}


In [12]:
# Setup notifications.
import lusid_notifications
import lusid_notifications.models as models
from lusidjam import RefreshingToken
from fbnsdkutilities import ApiClientFactory

notifications_api_factory = ApiClientFactory(
    lusid_notifications,
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="VSCode",
)

print('LUSID Environment Initialised')

email_notifcation_api = notifications_api_factory.build(lusid_notifications.SubscriptionsApi)

subscription_code = "ConstituentChange"
subscription_display_name = 'Constituent Change'
subscription_description = 'Consituent change subscription'

"""
create_subscription = {
    "id": {
        "scope": f"{scope}", "code": f"{subscription_code}"
        },
    "displayName": f'{subscription_display_name}',
    "description": f'{subscription_description}',
    "status": "Active",
    "matchingPattern": {
        "eventType": "Manual", "filter": "Message eq 'TestMessage'"
        },
    "useAsAuth": "Auth user",
}
"""

api_response = email_notifcation_api.create_subscription(
    models.CreateSubscription(
        id = models.ResourceId(
            scope = scope,
            code = subscription_code
        ),
        display_name = subscription_display_name,
        description = subscription_description,
        status = 'Active',
        matching_pattern = models.MatchingPattern(
            event_type = "ConstituentsUpdated"
        )
    )
)
pp.pprint(api_response)


LUSID Environment Initialised
{'created_at': datetime.datetime(2024, 6, 11, 13, 47, 25, 813139, tzinfo=tzutc()),
 'created_by': '00uv7uac52NTBxe1e2p7',
 'description': 'Consituent change subscription',
 'display_name': 'Constituent Change',
 'id': {'code': 'ConstituentChange', 'scope': 'Demo2'},
 'last_modified_at': datetime.datetime(2024, 6, 11, 13, 47, 25, 813139, tzinfo=tzutc()),
 'last_modified_by': '00uv7uac52NTBxe1e2p7',
 'matching_pattern': {'event_type': 'ConstituentsUpdated', 'filter': None},
 'notifications': None,
 'status': 'Active',
 'use_as_auth': '00uv7uac52NTBxe1e2p7'}


In [13]:
# Create notification.

notification_api = notifications_api_factory.build(lusid_notifications.NotificationsApi)

api_response = notification_api.create_email_notification(
    scope=scope,
    code=subscription_code,
    create_email_notification=models.CreateEmailNotification(
        description='Notification for when constituents change in ref portfolio.',
        subject='Constituents changed',
        plain_text_body='Constituents of the ref portfolio has changed.',
        email_address_to=['joe.cockings@finbourne.com']
    )
)


# Test change